In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

In [2]:
spark =  SparkSession.builder \
     .master('local') \
     .appName("Pyspark RDD Practises") \
     .getOrCreate()

In [3]:
sc = spark.sparkContext

In [4]:
file = sc.textFile("data/emp_huge.csv",3)

In [5]:
type(file)

pyspark.rdd.RDD

In [6]:
file.getNumPartitions()

3

In [7]:
file.sample(False,0.000035).collect()

['36629,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,,30.0',
 '115800,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,,10.0',
 '136449,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,,30.0',
 '146740,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,,10.0',
 '230609,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0',
 '231301,7788.0,SCOTT,ANALYST,7566.0,1982-12-09,3000.0,,20.0',
 '291036,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0',
 '308457,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0',
 '348135,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,,30.0',
 '413991,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,,30.0',
 '430464,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,,10.0',
 '445306,7839.0,KING,PRESIDENT,,1981-11-17,5000.0,,10.0',
 '463506,7839.0,KING,PRESIDENT,,1981-11-17,5000.0,,10.0',
 '516058,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0',
 '517152,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,,10.0',
 '532335,7934.0,MILLER,CLERK,7782.0,1982-01-2

In [8]:
#Function return tuple if valid value else return None
#if you are converting empty string to float then it gives error(couldn't convert string to float)
def each_line_to_tuple(lines):
    c = lines.split(',') 
    t = None
    def convert_string_float(x):
        try:
            x = float(x)
        except:
            x = 0.00
        return x
    def convert_string_int(x):
        try:
            x = int(float(x))
        except:
            print(x)
            x = 0
        return x
    try:
        if c[1]!='empno':
            t = (int(float(c[1])),c[2],c[3],convert_string_int(c[4]),c[5],float(c[6]),convert_string_float(c[7]),convert_string_int(c[8]))
    except:
        print(c)
    return t

In [9]:
#Convert each value in index to tuple and then remove None values from pipelined RDD
file_map = file.map(each_line_to_tuple)
filter_none = file_map.filter(lambda x:x!=None)
filter_none.take(5)


[(7499, 'ALLEN', 'SALESMAN', 7698, '1981-02-20', 1600.0, 300.0, 30),
 (7521, 'WARD', 'SALESMAN', 7698, '1981-02-22', 1250.0, 500.0, 30),
 (7566, 'JONES', 'MANAGER', 7839, '1981-04-02', 2975.0, 0.0, 20),
 (7654, 'MARTIN', 'SALESMAN', 7698, '1981-09-28', 1250.0, 1400.0, 30),
 (7698, 'BLAKE', 'MANAGER', 7839, '1981-05-01', 2850.0, 0.0, 30)]

In [10]:
filter_30 = filter_none.filter(lambda row: row[7] == (30))

In [11]:
filter_30.take(10)

[(7499, 'ALLEN', 'SALESMAN', 7698, '1981-02-20', 1600.0, 300.0, 30),
 (7521, 'WARD', 'SALESMAN', 7698, '1981-02-22', 1250.0, 500.0, 30),
 (7654, 'MARTIN', 'SALESMAN', 7698, '1981-09-28', 1250.0, 1400.0, 30),
 (7698, 'BLAKE', 'MANAGER', 7839, '1981-05-01', 2850.0, 0.0, 30),
 (7844, 'TURNER', 'SALESMAN', 7698, '1981-09-08', 1500.0, 0.0, 30),
 (7900, 'JAMES', 'CLERK', 7698, '1981-12-03', 950.0, 0.0, 30),
 (7499, 'ALLEN', 'SALESMAN', 7698, '1981-02-20', 1600.0, 300.0, 30),
 (7521, 'WARD', 'SALESMAN', 7698, '1981-02-22', 1250.0, 500.0, 30),
 (7654, 'MARTIN', 'SALESMAN', 7698, '1981-09-28', 1250.0, 1400.0, 30),
 (7698, 'BLAKE', 'MANAGER', 7839, '1981-05-01', 2850.0, 0.0, 30)]

In [12]:
df = spark.createDataFrame(filter_none)

In [13]:
df.printSchema()

root
 |-- _1: long (nullable = true)
 |-- _2: string (nullable = true)
 |-- _3: string (nullable = true)
 |-- _4: long (nullable = true)
 |-- _5: string (nullable = true)
 |-- _6: double (nullable = true)
 |-- _7: double (nullable = true)
 |-- _8: long (nullable = true)



In [14]:
df.show()

+----+------+---------+----+----------+------+------+---+
|  _1|    _2|       _3|  _4|        _5|    _6|    _7| _8|
+----+------+---------+----+----------+------+------+---+
|7499| ALLEN| SALESMAN|7698|1981-02-20|1600.0| 300.0| 30|
|7521|  WARD| SALESMAN|7698|1981-02-22|1250.0| 500.0| 30|
|7566| JONES|  MANAGER|7839|1981-04-02|2975.0|   0.0| 20|
|7654|MARTIN| SALESMAN|7698|1981-09-28|1250.0|1400.0| 30|
|7698| BLAKE|  MANAGER|7839|1981-05-01|2850.0|   0.0| 30|
|7782| CLARK|  MANAGER|7839|1981-06-09|2450.0|   0.0| 10|
|7788| SCOTT|  ANALYST|7566|1982-12-09|3000.0|   0.0| 20|
|7839|  KING|PRESIDENT|   0|1981-11-17|5000.0|   0.0| 10|
|7844|TURNER| SALESMAN|7698|1981-09-08|1500.0|   0.0| 30|
|7876| ADAMS|    CLERK|7788|1983-01-12|1100.0|   0.0| 20|
|7900| JAMES|    CLERK|7698|1981-12-03| 950.0|   0.0| 30|
|7902|  FORD|  ANALYST|7566|1981-12-03|3000.0|   0.0| 20|
|7934|MILLER|    CLERK|7782|1982-01-23|1300.0|   0.0| 10|
|7369| SMITH|    CLERK|7902|1980-12-17| 800.0|   0.0| 20|
|7499| ALLEN| 

In [15]:
emp_rdd = sc.parallelize(filter_none.take(20),3)

In [16]:
#get total salary for each deptno
emp_key_value = emp_rdd.map(lambda row:(row[7],row[5])).reduceByKey(lambda x,y:x+y)
emp_key_value.collect()

[(30, 16350.0), (10, 11200.0), (20, 13850.0)]

In [20]:
def partition_count(partition):
    total_sal = 0.0
    for row in partition:
        total_sal += float(row[5])
    yield  'Partition Total Sal:{0}'.format(total_sal)

In [18]:
emp_rdd.glom().collect()

[[(7499, 'ALLEN', 'SALESMAN', 7698, '1981-02-20', 1600.0, 300.0, 30),
  (7521, 'WARD', 'SALESMAN', 7698, '1981-02-22', 1250.0, 500.0, 30),
  (7566, 'JONES', 'MANAGER', 7839, '1981-04-02', 2975.0, 0.0, 20),
  (7654, 'MARTIN', 'SALESMAN', 7698, '1981-09-28', 1250.0, 1400.0, 30),
  (7698, 'BLAKE', 'MANAGER', 7839, '1981-05-01', 2850.0, 0.0, 30),
  (7782, 'CLARK', 'MANAGER', 7839, '1981-06-09', 2450.0, 0.0, 10)],
 [(7788, 'SCOTT', 'ANALYST', 7566, '1982-12-09', 3000.0, 0.0, 20),
  (7839, 'KING', 'PRESIDENT', 0, '1981-11-17', 5000.0, 0.0, 10),
  (7844, 'TURNER', 'SALESMAN', 7698, '1981-09-08', 1500.0, 0.0, 30),
  (7876, 'ADAMS', 'CLERK', 7788, '1983-01-12', 1100.0, 0.0, 20),
  (7900, 'JAMES', 'CLERK', 7698, '1981-12-03', 950.0, 0.0, 30),
  (7902, 'FORD', 'ANALYST', 7566, '1981-12-03', 3000.0, 0.0, 20)],
 [(7934, 'MILLER', 'CLERK', 7782, '1982-01-23', 1300.0, 0.0, 10),
  (7369, 'SMITH', 'CLERK', 7902, '1980-12-17', 800.0, 0.0, 20),
  (7499, 'ALLEN', 'SALESMAN', 7698, '1981-02-20', 1600.0, 30

In [21]:
each_part_count = emp_rdd.mapPartitions(partition_count)
each_part_count.collect()

['Partition Total Sal:12375.0',
 'Partition Total Sal:14550.0',
 'Partition Total Sal:14475.0']

In [22]:
def partition_with_idx_count(index,partition):
    total_sal = 0.0
    for row in partition:
        total_sal += float(row[5])
    yield  'Partition {0} Total Sal:{1}'.format(index,total_sal)

In [23]:
each_part_count = emp_rdd.mapPartitionsWithIndex(partition_with_idx_count)
each_part_count.collect()

['Partition 0 Total Sal:12375.0',
 'Partition 1 Total Sal:14550.0',
 'Partition 2 Total Sal:14475.0']

In [24]:
emp_rdd_distinct = emp_rdd.distinct()

In [25]:
emp_rdd.intersection(emp_rdd_distinct).collect()

[(7782, 'CLARK', 'MANAGER', 7839, '1981-06-09', 2450.0, 0.0, 10),
 (7934, 'MILLER', 'CLERK', 7782, '1982-01-23', 1300.0, 0.0, 10),
 (7499, 'ALLEN', 'SALESMAN', 7698, '1981-02-20', 1600.0, 300.0, 30),
 (7844, 'TURNER', 'SALESMAN', 7698, '1981-09-08', 1500.0, 0.0, 30),
 (7902, 'FORD', 'ANALYST', 7566, '1981-12-03', 3000.0, 0.0, 20),
 (7521, 'WARD', 'SALESMAN', 7698, '1981-02-22', 1250.0, 500.0, 30),
 (7566, 'JONES', 'MANAGER', 7839, '1981-04-02', 2975.0, 0.0, 20),
 (7698, 'BLAKE', 'MANAGER', 7839, '1981-05-01', 2850.0, 0.0, 30),
 (7788, 'SCOTT', 'ANALYST', 7566, '1982-12-09', 3000.0, 0.0, 20),
 (7876, 'ADAMS', 'CLERK', 7788, '1983-01-12', 1100.0, 0.0, 20),
 (7839, 'KING', 'PRESIDENT', 0, '1981-11-17', 5000.0, 0.0, 10),
 (7654, 'MARTIN', 'SALESMAN', 7698, '1981-09-28', 1250.0, 1400.0, 30),
 (7900, 'JAMES', 'CLERK', 7698, '1981-12-03', 950.0, 0.0, 30),
 (7369, 'SMITH', 'CLERK', 7902, '1980-12-17', 800.0, 0.0, 20)]

In [26]:
emp_rdd.union(emp_rdd_distinct)

UnionRDD[39] at union at NativeMethodAccessorImpl.java:0

In [27]:
emp_rdd.map(lambda row:(row[7],row)).groupByKey().collect()

[(30, <pyspark.resultiterable.ResultIterable at 0x7f9d40ee3198>),
 (10, <pyspark.resultiterable.ResultIterable at 0x7f9d40ee3748>),
 (20, <pyspark.resultiterable.ResultIterable at 0x7f9d40f33c18>)]

In [28]:
#Each deptno total sum using reduceByKey
emp_rdd.map(lambda row: (row[7],row[5])).reduceByKey(lambda total,ele:total+ele).collect()

[(30, 16350.0), (10, 11200.0), (20, 13850.0)]

In [29]:
spark.stop()